In [ ]:
import os
import pandas as pd

import numpy as np

import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import models

from tensorflow.keras.models import load_model
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

from keras.constraints import maxnorm

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn import pipeline

from scipy.stats import shapiro, normaltest, anderson
from statsmodels.stats.diagnostic import lilliefors

from joblib import load

import seaborn as sns
from statsmodels.graphics.gofplots import qqplot
import matplotlib.pyplot as plt

In [ ]:
X_treinamento = pd.read_json(r'X_treinamento.json')
X_teste = pd.read_json      (r'X_teste.json')
treino_json = pd.read_json  (r'y_treinamento.json', typ='series')
teste_json = pd.read_json   (r'y_teste.json', typ='series')
X_validacao = pd.read_json  (r'X_validacao.json')
y_validacao = pd.read_json  (r'y_validacao.json', typ='series')

In [ ]:
a = []
b = []

for i in range(y_validacao.size):
    a.append(list(y_validacao[i]))

In [ ]:
y_treinamento = np.array(a, dtype='float32')
y_teste = np.array(b, dtype='float32')

In [ ]:
y = np.array(a, dtype='float32')

In [ ]:
def calcula12x12CUC(matrix):
    
    df = pd.DataFrame(np.zeros((16, 16)))
    
    k = 4
    w = 4
    for i in range(0,12):
        if w > 14:
            w = 4
        for j in range(0,12):
            df.at[i,j] = 0 + matrix[k,w]
            w = w + 1
        k = k+1

    k = 4
    w = 0
    for i in range(0,12):
        if w > 11:
            w = 0
        for j in range(4,16):
            df.at[i,j] = df.at[i,j] + matrix[k,w]
            w = w + 1
        k = k + 1

    k = 0
    w = 4
    for i in range(4,16):
        if w > 14:
            w = 4
        for j in range(0,12):
            df.at[i,j] = df.at[i,j] + matrix[k,w]
            w = w + 1
        k = k + 1

    k = 0
    w = 0
    for i in range(4,16):
        if w > 11:
            w = 0
        for j in range(4,16):
            df.at[i,j] = df.at[i,j] + matrix[k,w]
            w = w + 1
        k = k + 1

    soma = 0.0
    media = df.values.mean()

    for i in range(4, 12):
        for j in range(4,12):
            valor = df.at[i,j]
            valor_abs = abs(valor-media)
            final = valor_abs/(256*media)
            soma = soma+final

    cuc = (1.0-soma)*100.0

    return cuc

In [ ]:
def calcula12x15CUC(matrix):
    
    df = pd.DataFrame(np.zeros((16, 16)))
    
    k = 4
    w = 4
    for i in range(0,12):
        if w > 14:
            w = 4
        for j in range(0,12):
            df.at[i,j] = 0 + matrix[k,w]
            w = w + 1
        k = k+1

    k = 4
    w = 0
    for i in range(0,12):
        if w > 11:
            w = 0
        for j in range(4,16):
            df.at[i,j] = df.at[i,j] + matrix[k,w]
            w = w + 1
        k = k + 1

    k = 0
    w = 4
    for i in range(6,16):
        if w > 14:
            w = 4
        for j in range(0,12):
            df.at[i,j] = df.at[i,j] + matrix[k,w]
            w = w + 1
        k = k + 1

    k = 0
    w = 0
    for i in range(6,16):
        if w > 11:
            w = 0
        for j in range(4,16):
            df.at[i,j] = df.at[i,j] + matrix[k,w]
            w = w + 1
        k = k + 1

    soma = 0.0
    media = df.values.mean()

    for i in range(4, 12):
        for j in range(4,12):
            valor = df.at[i,j]
            valor_abs = abs(valor-media)
            final = valor_abs/(256*media)
            soma = soma+final

    cuc = (1.0-soma)*100.0
    
    return cuc

In [ ]:
def calcula15x15CUC(matrix):
    
    df = pd.DataFrame(np.zeros((16, 16)))
    
    k = 5
    w = 5
    for i in range(0,11):
        if w > 11:
            w = 5
        for j in range(0,11):
            df.at[i,j] = 0 + matrix[k,w]
            w = w + 1
        k = k+1

    k = 5
    w = 0
    for i in range(0,11):
        if w > 10:
            w = 0
        for j in range(5,16):
            df.at[i,j] = df.at[i,j] + matrix[k,w]
            w = w + 1
        k = k + 1

    k = 0
    w = 5
    for i in range(5,16):
        if w > 11:
            w = 5
        for j in range(0,11):
            df.at[i,j] = df.at[i,j] + matrix[k,w]
            w = w + 1
        k = k + 1

    k = 0
    w = 0
    for i in range(5,16):
        if w > 10:
            w = 0
        for j in range(5,16):
            df.at[i,j] = df.at[i,j] + matrix[k,w]
            w = w + 1
        k = k + 1

    soma = 0.0
    media = df.values.mean()

    for i in range(4, 12):
        for j in range(4,12):
            valor = df.at[i,j]
            valor_abs = abs(valor-media)
            final = valor_abs/(256*media)
            soma = soma+final

    cuc = (1.0-soma)*100.0
    
    return cuc

In [ ]:
def build_model(
    optimizer,
    learning_rate,
    activation,
    init_mode,
    dropout_rate,
    weight_constraint,
    neurons,
    regularizers
):
    model = models.Sequential()
    
    # layer 1
    model.add(
        layers.Dense(
            neurons,
            activation = activation,
            kernel_initializer=init_mode,
            kernel_constraint=maxnorm(weight_constraint),
            kernel_regularizer=regularizers,
            input_shape=[len(X_validacao.keys())]
        )
    )
    model.add(layers.Dropout(dropout_rate))
    
    # layer 2
    model.add(layers.Dense(256))
    
    model.compile(
        loss='mse',
        optimizer=optimizer(learning_rate),
        metrics=['mae', 'mse', keras.metrics.RootMeanSquaredError()]
    )
    
    return model

In [ ]:
loaded_model = KerasRegressor(build_fn=build_model, verbose=1)

In [ ]:
loaded_model.model = load_model(r'model-70-30.h5')

In [ ]:
scaler = load(r'scaler-70-30.joblib')

In [ ]:
loaded_steps = [('scaler', scaler), 
         ('estimator', loaded_model)]

In [ ]:
loaded_pipe = pipeline.Pipeline(loaded_steps)

In [ ]:
plm = loaded_pipe.predict(X_validacao)

In [ ]:
predictedCuc12x12 = []
for i in range(plm.shape[0]):
    pCuc12x12 = calcula12x12CUC(plm[i].reshape(16,16))
    predictedCuc12x12.append(pCuc12x12)

In [ ]:
expectedCuc12x12 = []
for i in range(y.shape[0]):
    eCuc12x12 = calcula12x12CUC(y[i].reshape(16,16)) 
    expectedCuc12x12.append(eCuc12x12)

In [ ]:
predictedCuc12x15 = []
for i in range(plm.shape[0]):
    pCuc12x15 = calcula12x15CUC(plm[i].reshape(16,16))
    predictedCuc12x15.append(pCuc12x15)

In [ ]:
expectedCuc12x15 = []
for i in range(y.shape[0]):
    eCuc12x15 = calcula12x15CUC(y[i].reshape(16,16)) 
    expectedCuc12x15.append(eCuc12x15)

In [ ]:
predictedCuc15x15 = []
for i in range(plm.shape[0]):
    pCuc15x15 = calcula15x15CUC(plm[i].reshape(16,16))
    predictedCuc15x15.append(pCuc15x15)

In [ ]:
expectedCuc15x15 = []
for i in range(y.shape[0]):
    eCuc15x15 = calcula15x15CUC(y[i].reshape(16,16)) 
    expectedCuc15x15.append(eCuc15x15)

In [ ]:
for prediction, real, exp in zip(predictedCuc12x12, expectedCuc12x12, experimentos):
    print("Experimento: %i - Prediction: %f - Real: %f - Error: %f" % (exp, prediction, real, prediction - real))

In [ ]:
for prediction, real, exp in zip(predictedCuc12x15, expectedCuc12x15, experimentos):
    print("Experimento: %i - Prediction: %f - Real: %f - Error: %f" % (exp, prediction, real, prediction - real))

In [ ]:
for prediction, real, exp in zip(predictedCuc15x15, expectedCuc15x15, experimentos):
    print("Experimento: %i - Prediction: %f - Real: %f - Error: %f" % (exp, prediction, real, prediction - real))

In [ ]:
# 12x12

In [ ]:
print("CUC 12x12 MAE:", "{0:0.5f}".format(mean_absolute_error(predictedCuc12x12, expectedCuc12x12)))

In [ ]:
print("CUC 12x12 MSE: ", "{0:0.5f}".format(mean_squared_error(predictedCuc12x12, expectedCuc12x12)))

In [ ]:
print("CUC 12x12 RMSE:", "{0:0.5f}".format(mean_squared_error(predictedCuc12x12, expectedCuc12x12, squared=False)))

In [ ]:
print("CUC 12x12 R-squared: ", r2_score(predictedCuc12x12, expectedCuc12x12))

In [ ]:
# 12x15

In [ ]:
print("CUC 12x15 MAE:", "{0:0.5f}".format(mean_absolute_error(predictedCuc12x15, expectedCuc12x15)))

In [ ]:
print("CUC 12x15 MSE: ", "{0:0.5f}".format(mean_squared_error(predictedCuc12x15, expectedCuc12x15)))

In [ ]:
print("CUC 12x15 RMSE:", "{0:0.5f}".format(mean_squared_error(predictedCuc12x15, expectedCuc12x15, squared=False)))

In [ ]:
print("CUC 12x15 R-squared: ", r2_score(predictedCuc12x15, expectedCuc12x15))

In [ ]:
#15x15

In [ ]:
print("CUC 15x15 MAE:", "{0:0.5f}".format(mean_absolute_error(predictedCuc15x15, expectedCuc15x15)))

In [ ]:
print("CUC 15x15 MSE: ", "{0:0.5f}".format(mean_squared_error(predictedCuc15x15, expectedCuc15x15)))

In [ ]:
print("CUC 15x15 RMSE:", "{0:0.5f}".format(mean_squared_error(predictedCuc15x15, expectedCuc15x15, squared=False)))

In [ ]:
print("CUC 15x15 R-squared: ", r2_score(predictedCuc15x15, expectedCuc15x15))

In [ ]:
hist = pd.read_json(r'history.json')

In [ ]:
sns.set_theme()

In [ ]:
def plot_history(hist):
#     hist = pd.DataFrame(grid_result.best_estimator_['estimator'].model.history.history)
    hist.reset_index(inplace=True)
    hist = hist.rename(columns = {'index':'epoch'})

    plt.figure()
    plt.xlabel('Época')
    plt.ylabel('Erro [MSE]')
    plt.plot(hist['epoch'], hist['mse'],
           label='Erro')
    plt.ylim([0,2])
    plt.legend()


plot_history(hist)

In [ ]:
error12x12 = pd.Series(np.array(predictedCuc12x12) - np.array(expectedCuc12x12))
error12x15 = pd.Series(np.array(predictedCuc12x15) - np.array(expectedCuc12x15))
error15x15 = pd.Series(np.array(predictedCuc15x15) - np.array(expectedCuc15x15))

In [ ]:
error12x12.to_json(r'error12x12.json')
error12x15.to_json(r'error12x15.json')
error15x15.to_json(r'error15x15.json')

In [ ]:
plt.hist(error12x12, bins = 15)
plt.xlabel("Resíduos")
_ = plt.ylabel("Observações")

In [ ]:
plt.hist(error12x15, bins = 15)
plt.xlabel("Resíduos")
_ = plt.ylabel("Observações")

In [ ]:
plt.hist(error15x15, bins = 15)
plt.xlabel("Resíduos")
_ = plt.ylabel("Observações")

In [ ]:
resultados12x12 = pd.DataFrame({'Valores Preditos':predictedCuc12x12, 'Resíduos':error12x12})
resultados12x15 = pd.DataFrame({'Valores Preditos':predictedCuc12x15, 'Resíduos':error12x15})
resultados15x15 = pd.DataFrame({'Valores Preditos':predictedCuc15x15, 'Resíduos':error15x15})

In [ ]:
sns.scatterplot(data=resultados12x12, x="Valores Preditos", y="Resíduos")
ax = sns.lineplot(x=resultados12x12['Valores Preditos'],y=0, dashes=[(2, 2), (2, 2)])
ax.lines[0].set_linestyle("--")
plt.show()

In [ ]:
sns.scatterplot(data=resultados12x15, x="Valores Preditos", y="Resíduos")
ax = sns.lineplot(x=resultados12x15['Valores Preditos'],y=0, dashes=[(2, 2), (2, 2)])
ax.lines[0].set_linestyle("--")
plt.show()

In [ ]:
sns.scatterplot(data=resultados15x15, x="Valores Preditos", y="Resíduos")
ax = sns.lineplot(x=resultados15x15['Valores Preditos'],y=0, dashes=[(2, 2), (2, 2)])
ax.lines[0].set_linestyle("--")
plt.show()

In [ ]:
qqplot(error12x12, line='s')
plt.show()

In [ ]:
qqplot(error12x15, line='s')
plt.show()

In [ ]:
qqplot(error15x15, line='s')
plt.show()

In [ ]:
sp12x12 = shapiro(error12x12)
print("SP 12x12:", sp12x12)

In [ ]:
nt12x12 = normaltest(error12x12)
print("NT 12x12:", nt12x12)

In [ ]:
lf12x12 = lilliefors(error12x12)
print("LF 12x12:", lf12x12)

In [ ]:
result12x12 = anderson(error12x12)
print('Statistic: %.3f' % result12x12.statistic)
for i in range(len(result12x12.critical_values)):
    sl12x12, cv12x12 = result12x12.significance_level[i], result12x12.critical_values[i]
    if result12x12.statistic < result12x12.critical_values[i]:
        print('%.3f: %.3f, data looks normal (fail to reject H0)' % (sl12x12, cv12x12))
    else:
        print('%.3f: %.3f, data does not look normal (reject H0)' % (sl12x12, cv12x12))

In [ ]:
print("AD 12x12:", result12x12)

In [ ]:
#12x15

In [ ]:
sp12x15 = shapiro(error12x15)
print("SP 12x15:", sp12x15)

In [ ]:
nt12x15 = normaltest(error12x15)
print("NT 12x15:", nt12x15)

In [ ]:
lf12x15 = lilliefors(error12x15)
print("LF 12x15:", lf12x15)

In [ ]:
result12x15 = anderson(error12x15)
print('Statistic: %.3f' % result12x15.statistic)
for i in range(len(result12x15.critical_values)):
    sl12x15, cv12x15 = result12x15.significance_level[i], result12x15.critical_values[i]
    if result12x15.statistic < result12x15.critical_values[i]:
        print('%.3f: %.3f, data looks normal (fail to reject H0)' % (sl12x15, cv12x15))
    else:
        print('%.3f: %.3f, data does not look normal (reject H0)' % (sl12x15, cv12x15))

In [ ]:
print("AD 12x15:", result12x15)

In [ ]:
#15x15

In [ ]:
sp15x15 = shapiro(error15x15)
print("SP 15x15:", sp15x15)

In [ ]:
nt15x15 = normaltest(error15x15)
print("NT 15x15:", nt15x15)

In [ ]:
lf15x15 = lilliefors(error15x15)
print("LF 15x15:", lf15x15)

In [ ]:
result15x15 = anderson(error15x15)
print('Statistic: %.3f' % result15x15.statistic)
for i in range(len(result15x15.critical_values)):
    sl15x15, cv15x15 = result15x15.significance_level[i], result15x15.critical_values[i]
    if result15x15.statistic < result15x15.critical_values[i]:
        print('%.3f: %.3f, data looks normal (fail to reject H0)' % (sl15x15, cv15x15))
    else:
        print('%.3f: %.3f, data does not look normal (reject H0)' % (sl15x15, cv15x15))

In [ ]:
print("AD 12x15:", result15x15)